In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


import matplotlib
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
matplotlib.rcParams['axes.unicode_minus'] = False   

## 서울시_동별_인구수

In [ ]:
# 서울시 구별 인구수 통합
gu_list = ['종로구','중구','용산구','성동구','광진구','동대문구','중랑구','성북구','강북구','도봉구','노원구','은평구','서대문구','마포구','양천구','강서구','구로구','금천구','영등포구','동작구','관악구','서초구','강남구','송파구','강동구']
data = pd.DataFrame()
for gu in gu_list:
    temp_data = pd.read_csv('{}_동별인구수.csv'.format(gu),encoding='cp949')
    data = pd.concat([data,temp_data])

In [ ]:
# 총인구수 항목만 추출
data1 = data[data['항목']=='총인구수 (명)']

In [ ]:
# -구 데이터 제거
index = []
for i in data1['행정구역(동읍면)별']:
    index.append(i.endswith('구')==False)
    
data2 = data1[index].reset_index(drop=True)

In [ ]:
data2.columns

Index(['행정구역(동읍면)별', '항목', '계', '0 - 4세', '5 - 9세', '10 - 14세', '15 - 19세',
       '20 - 24세', '25 - 29세', '30 - 34세', '35 - 39세', '40 - 44세', '45 - 49세',
       '50 - 54세', '55 - 59세', '60 - 64세', '65 - 69세', '70 - 74세', '75 - 79세',
       '80 - 84세', '85 - 89세', '90 - 94세', '95 - 99세', '100+'],
      dtype='object')

In [ ]:
final_data = data2[['행정구역(동읍면)별','계']]
final_data.columns = ['행정구역','인구수']

In [ ]:
final_data.to_csv('서울시_동별인구수_최종.csv',index=False)

## 서울시_동별_폐기물

In [ ]:
# % -> 소수점 변경
recycle = pd.read_csv('서울시_자치구별_재활용률_최종.csv',encoding='utf-8')
for i in range(len(recycle)):
    recycle.loc[i,'재활용률'] = int(recycle['재활용률'][i][:2])/100
recycle.to_csv('서울시_자치구별_재활용률_찐최종.csv')

In [ ]:
# 구별 폐기물 발생량,재활용량 -> 동별 
gu_list = ['종로구','중구','용산구','성동구','광진구','동대문구','중랑구','성북구','강북구','도봉구','노원구','은평구','서대문구','마포구','양천구','강서구','구로구','금천구','영등포구','동작구','관악구','서초구','강남구','송파구','강동구']
recycle = pd.read_csv('서울시_자치구별_재활용률_찐최종.csv',encoding='utf-8')
final_df = pd.DataFrame()
for gu in gu_list:
    temp_data = pd.read_csv('{}_동별인구수.csv'.format(gu),encoding='cp949')
    temp_data = temp_data[temp_data['항목']=='총인구수 (명)']
    dong_list = temp_data['행정구역(동읍면)별'].unique().tolist()[1:]
    if '신사동' in dong_list:         #신사동 중복 -> 신사동 포함 구 출력 : 관악구, 강남구 
        print(gu,dong_list)
    구_인구 = temp_data.loc[temp_data['행정구역(동읍면)별']==gu,'계'].values[0]
    구발생량 = recycle.loc[recycle['자치구별']==gu,'발생량'].values[0]
    구재활용량 = recycle.loc[recycle['자치구별']==gu,'재활용량'].values[0]
    일인당발생량 = 구발생량 / 구_인구
    일인당재활용량 = 구재활용량 / 구_인구
    동발생량_list = [일인당발생량 * temp_data.loc[temp_data['행정구역(동읍면)별']==dong,'계'].values[0] for dong in dong_list]
    동재활용량_list = [일인당재활용량 * temp_data.loc[temp_data['행정구역(동읍면)별']==dong,'계'].values[0] for dong in dong_list]
    
    
    #동재활용률_list = [round(동재활용량/동발생량,2) for 동발생량,동재활용량 in zip(동발생량_list,동재활용량_list)]
    temp_df = pd.DataFrame({'시군구' : gu,
                            '행정구역' : dong_list,
                           '발생량' : 동발생량_list,
                           '재활용량' : 동재활용량_list,
                           })
    final_df = pd.concat([final_df,temp_df])
final_df.reset_index(drop=True,inplace=True)
final_df.to_csv('서울시_동별_폐기물_최종.csv',index=False,encoding='euc-kr')

관악구 ['보라매동', '청림동', '성현동', '행운동', '낙성대동', '청룡동', '은천동', '중앙동', '인헌동', '남현동', '서원동', '신원동', '서림동', '신사동', '신림동', '난향동', '조원동', '대학동', '삼성동', '미성동', '난곡동']
강남구 ['신사동', '논현1동', '논현2동', '압구정동', '청담동', '삼성1동', '삼성2동', '대치1동', '대치2동', '대치4동', '역삼1동', '역삼2동', '도곡1동', '도곡2동', '개포1동', '개포2동', '개포4동', '세곡동', '일원본동', '일원1동', '일원2동', '수서동']


## 주택종류별 데이터

In [ ]:
df = pd.read_csv('서울시_최종.csv',encoding='cp949')
df.loc[(df['시군구']=='관악구') & (df['행정구역']=='신사동'),'행정구역'] = '관악구_신사동'
df.loc[(df['시군구']=='강남구') & (df['행정구역']=='신사동'),'행정구역'] = '강남구_신사동'

In [ ]:
data = pd.read_csv('건물용도별개수.csv',encoding='cp949')
data.rename(columns={'행정구역' : '이거아님',
                     'ADM_NM' : '행정구역'},inplace=True)

data.loc[(data['시군구']=='관악구') & (data['행정구역']=='신사동'),'행정구역'] = '관악구_신사동'
data.loc[(data['시군구']=='강남구') & (data['행정구역']=='신사동'),'행정구역'] = '강남구_신사동'


In [ ]:
#세부용도명 결측값 제거 
data = data[data['세부용도명'].isnull()==False].reset_index(drop=True)

In [ ]:
# 상업용, 주거용 변수 추가
for i in ['상업용','주거용']:
    temp_data = data.loc[data['건물용도분류명']==i].reset_index(drop=True).groupby(by='행정구역').count()
    df = pd.merge(df,temp_data.iloc[:,1],on='행정구역',how='left')
    


In [ ]:
# 주택 변수 추가
주택_list = [x for x in data['세부용도명'].unique().tolist() if x.endswith('주택')]
for i in 주택_list:
    temp_data = data.loc[data['세부용도명']==i].reset_index(drop=True).groupby(by='행정구역').count()
    df = pd.merge(df,temp_data.iloc[:,1],on='행정구역',how='left')

In [ ]:
# 아파트 변수 추가
temp_data = data.loc[data['세부용도명']=='아파트'].reset_index(drop=True).groupby(by='행정구역').count()
df = pd.merge(df,temp_data.iloc[:,1],on='행정구역',how='left') 

In [ ]:
# df.columns =['시군구', '행정구역', '인구수', '가구수', '세대당인구수', '일인가구수', '기초수급가구수', '소득분위',
#        '재활용량', '쓰레기발생량', '상업용', '주거용', '단독주택', '다가구주택', '다세대주택', 
#              '다중주택', '연립주택', '공동주택', '아파트']


In [ ]:
# 결측값 처리 -> 0
df = df.fillna(0)

In [ ]:
# 자료형 변경 -> float
df.iloc[:,2:] = df.iloc[:,2:].astype('float')

In [ ]:
# 연립주택, 아파트에 분리수거함 있다고 가정
df['분리수거함개수'] =df['연립주택'] + df['아파트'] 

In [ ]:
df.to_csv('서울시_최종_주택종류추가.csv',encoding='cp949',index=False)

In [ ]:
df_ = df.copy()
df_.drop(['단독주택','다가구주택','다세대주택','다중주택','연립주택','공동주택','아파트'],axis=1,inplace=True)
df_.to_csv('서울시_최종.csv',encoding='cp949',index=False)

In [ ]:
data = pd.read_csv('서울시_최종.csv',encoding='cp949')
data.to_csv('서울시_최종.csv',encoding='cp949',index=False)

In [ ]:
df_

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
df[X_col] = scaler.fit_transform(scaler)

In [ ]:
df['행정구역']

## 서울시 외식업

In [ ]:
#사용안함
#서울시 외식업 데이터
store = pd.read_csv('서울시_외식업.csv',encoding='utf-8')

#동 데이터만 추출
idx = store.isnull()['행정구역']==False
store = store.loc[idx,['행정구역','전체점포수']].reset_index(drop=True)
store['전체점포수'] = [x.replace(',','') for x in store['전체점포수'].tolist()]
store.rename(columns = {'전체점포수' : '외식업점포수'},inplace=True)
store.to_csv('서울시_외식업_최종.csv',encoding='cp949',index=False)


## 서울시 1인가구

In [ ]:
data = pd.read_csv('서울시_1인가구수_전처리완료.csv',encoding='utf-8')
data = data.replace('X',np.nan)[['행정구역','1인가구']]
data = data.loc[data['행정구역'] !='소계']
data['1인가구'] =[x.replace(',','') for x in data['1인가구']]
data.to_csv('서울시_1인가구수_최종.csv',encoding='cp949',index=False)

## 서울시 지하철역 수

In [ ]:
#사용안함
subway = pd.read_csv('서울시_지하철역_동별_개수.csv',encoding='cp949')
subway = subway[['ADM_NM','NUMPOINTS']]
subway.rename(columns = {'ADM_NM' : '행정구역',
                        'NUMPOINTS' : '지하철역수'},inplace=True)
subway['행정구역'] = [x.replace('·','.') for x in subway['행정구역']]
subway.to_csv('서울시_지하철역수_최종.csv',encoding='cp949',index=False)

## 서울시 버스정류장 수

In [ ]:
#사용안함
busstop = pd.read_csv('서울시_버스정류장_동별개수.csv',encoding='cp949')
busstop = busstop[['ADM_NM','NUMPOINTS']]
busstop.rename(columns = {'ADM_NM' : '행정구역',
                        'NUMPOINTS' : '버스정류장수'},inplace=True)
busstop['행정구역'] = [x.replace('·','.') for x in busstop['행정구역']]
busstop.to_csv('서울시_버스정류장수_최종.csv',encoding='cp949',index=False)

## 서울시 초등학교 수

In [ ]:
#사용안함
e_school = pd.read_csv('서울시_초등학교_동별개수.csv',encoding='cp949')
e_school = e_school[['ADM_NM','NUMPOINTS']]
e_school.rename(columns = {'ADM_NM' : '행정구역',
                        'NUMPOINTS' : '초등학교수'},inplace=True)
e_school['행정구역'] = [x.replace('·','.') for x in e_school['행정구역']]
e_school.to_csv('서울시_초등학교수_최종.csv',encoding='cp949',index=False)

## 서울시 소득분위

In [ ]:
income = pd.read_csv('서울시_소득분위.csv',encoding='cp949')
idx = [x.endswith('위') for x in income['소득분위']]
income = income[idx].reset_index(drop=True)
income['소득분위'] = [x[0] for x in income['소득분위']]
income['소득분위'] = income['소득분위'].astype('int')
income['가구수'] = [x.replace(',','') for x in income['가구수']]
income.to_csv('서울시_소득분위_최종.csv',encoding='cp949',index=False)

## 서울시 주택종류, 일반가구

In [ ]:
house = pd.read_csv('서울시_주택종류및일반가구_0806.csv',encoding='utf-8')
house.drop('자치구',axis=1,inplace=True)
house.loc[house['행정구역']=='종로5·6가동','행정구역'] = '종로5.6가동'
house.to_csv('서울시_주택종류및일반가구_최종.csv',encoding='cp949',index=False)

## 서울시 데이터 merge

In [ ]:
trash = pd.read_csv('서울시_동별_폐기물_최종.csv',encoding='euc-kr')
pop = pd.read_csv('서울시_동별인구수_최종.csv',encoding='utf-8')
store = pd.read_csv('서울시_외식업_최종.csv',encoding='cp949')
one = pd.read_csv('서울시_1인가구수_최종.csv',encoding='cp949')
old_pop = pd.read_csv('서울시세대.txt',sep='\t')
pool = pd.read_csv('서울시기초수급.txt',sep='\t')
house = pd.read_csv('서울시_주택종류및일반가구_최종.csv',encoding='cp949')
subway = pd.read_csv('서울시_지하철역수_최종.csv',encoding='cp949')
busstop = pd.read_csv('서울시_버스정류장수_최종.csv',encoding='cp949')
e_school = pd.read_csv('서울시_초등학교수_최종.csv',encoding='cp949')
income = pd.read_csv('서울시_소득분위_최종.csv',encoding='cp949')

In [ ]:
# pop 열 이름 변경

pop.columns = ['행정구역','인구수']

In [ ]:
# 기초수급 가구, 인원 
pool = pool[['동','총가구','총인원']]

# 동 추출
dong_idx = [idx for idx,dong in enumerate(pool['동'].tolist()) if dong.endswith('동')]

# 인덱스 초기화
pool = pool.loc[dong_idx].reset_index(drop=True)

# 천단위 (,) 제거
pool['총가구'] = [x.replace(',','') for x in pool['총가구'].tolist()]
pool['총인원'] = [x.replace(',','') for x in pool['총인원'].tolist()]

# 총가구, 총인원,고령자 데이터타입 변경 : object -> int
pool['총가구'] = pool['총가구'].astype('int')
pool['총인원'] = pool['총인원'].astype('int')


# 열 이름 변경
pool.rename(columns = {'동' : '행정구역',
                      '총가구' : '기초수급_가구',
                      '총인원' : '기초수급_인원'},inplace=True)



In [ ]:
# 세대당인구, 고령인구
old_pop = old_pop[['동','세대당인구','65세이상고령자']]
old_pop = old_pop.loc[(old_pop['동']!='소계') & (old_pop['동']!='합계')]  #소계, 합계 행 제거
old_pop.reset_index(drop=True,inplace=True)  #인덱스 초기화
old_pop.rename(columns= {'동' : '행정구역'},inplace= True) #열 이름 변경 : 동 -> 행정구역
old_pop['65세이상고령자'] = [x.replace(',','') for x in old_pop['65세이상고령자']]
old_pop['65세이상고령자'] = old_pop['65세이상고령자'].astype('int')

In [ ]:
# trash 데이터 '신사동' 데이터 값 변경
print([idx for idx,dong in enumerate(trash['행정구역'].tolist()) if dong=='신사동'])
print(trash['행정구역'][333])
trash.loc[332,'행정구역']='관악구_신사동'
trash.loc[358,'행정구역']='강남구_신사동'

In [ ]:
# pop 데이터 '신사동' 데이터 값 변경
print([idx for idx,dong in enumerate(pop['행정구역'].tolist()) if dong=='신사동'])
print(pop['행정구역'][333])
pop.loc[332,'행정구역']='관악구_신사동'
pop.loc[358,'행정구역']='강남구_신사동'

In [ ]:
# old_pop 데이터 '신사동' 데이터 값 변경
print([idx for idx,dong in enumerate(old_pop['행정구역'].tolist()) if dong=='신사동'])
print(old_pop['행정구역'][330])
old_pop.loc[329,'행정구역']='관악구_신사동'
old_pop.loc[358,'행정구역']='강남구_신사동'


In [ ]:
# pool 데이터 '신사동' 데이터 값 변경
print([idx for idx,dong in enumerate(pool['행정구역'].tolist()) if dong=='신사동'])
print(pool['행정구역'][330])
pool.loc[329,'행정구역']='관악구_신사동'
pool.loc[358,'행정구역']='강남구_신사동'

In [ ]:
# store 데이터 '신사동' 데이터 값 변경
print([idx for idx,dong in enumerate(store['행정구역'].tolist()) if dong=='신사동'])
print(store['행정구역'][332])
store.loc[331,'행정구역']='관악구_신사동'
store.loc[357,'행정구역']='강남구_신사동'

In [ ]:
# one 데이터 '신사동' 데이터 값 변경
print([idx for idx,dong in enumerate(one['행정구역'].tolist()) if dong=='신사동'])
print(one['행정구역'][330])
one.loc[329,'행정구역']='관악구_신사동'
one.loc[358,'행정구역']='강남구_신사동'

In [ ]:
# subway 데이터 '신사동' 데이터 값 변경
print([idx for idx,dong in enumerate(subway['행정구역'].tolist()) if dong=='신사동'])
print(subway['행정구역'][329])
subway.loc[328,'행정구역']='관악구_신사동'
subway.loc[357,'행정구역']='강남구_신사동'

In [ ]:
# busstop 데이터 '신사동' 데이터 값 변경
print([idx for idx,dong in enumerate(busstop['행정구역'].tolist()) if dong=='신사동'])
print(subway['행정구역'][329])
busstop.loc[328,'행정구역']='관악구_신사동'
busstop.loc[357,'행정구역']='강남구_신사동'

In [ ]:
# e_school 데이터 '신사동' 데이터 값 변경
print([idx for idx,dong in enumerate(e_school['행정구역'].tolist()) if dong=='신사동'])
print(e_school['행정구역'][329])
e_school.loc[328,'행정구역']='관악구_신사동'
e_school.loc[357,'행정구역']='강남구_신사동'

In [ ]:
# income 데이터 '신사동' 데이터 값 변경
print([idx for idx,dong in enumerate(income['행정구역'].tolist()) if dong=='신사동'])
print(income['행정구역'][332])
income.loc[331,'행정구역']='관악구_신사동'
income.loc[357,'행정구역']='강남구_신사동'

In [ ]:
# house 데이터 '신사동' 데이터 값 변경
print([idx for idx,dong in enumerate(house['행정구역'].tolist()) if dong=='신사동'])
print(house['행정구역'][329])
house.loc[328,'행정구역']='관악구_신사동'
house.loc[357,'행정구역']='강남구_신사동'

In [ ]:
# old_pop 데이터 '종로5·6가동' 데이터 값 변경 -> '종로5.6가동'
old_pop.loc[old_pop['행정구역']=='종로5·6가동','행정구역']='종로5.6가동'

In [ ]:
# pool 데이터 '종로5·6가동' 데이터 값 변경 -> '종로5.6가동'
pool.loc[pool['행정구역']=='종로5·6가동','행정구역']='종로5.6가동'

In [ ]:
# one 데이터 '종로5·6가동' 데이터 값 변경 -> '종로5.6가동'
one.loc[one['행정구역']=='종로5·6가동','행정구역']='종로5.6가동'

In [ ]:
# 동 이름 변경 -> ex) 제1동 -> 1동
tq = ['제{}동'.format(i) for i in range(1,10)]


for idx,dong in enumerate(trash['행정구역'].tolist()):
    if dong[-3:] in tq:
        dong = dong[:-3] + dong[-2:]
        trash.loc[idx,'행정구역'] = dong
        
        
for idx,dong in enumerate(pop['행정구역'].tolist()):
    if dong[-3:] in tq:
        dong = dong[:-3] + dong[-2:]
        pop.loc[idx,'행정구역'] = dong

In [ ]:
# 흥제1동,흥제2동,흥제3동 복구

for i in range(1,4):
    trash.loc[trash['행정구역']=='흥{}동'.format(i),'행정구역'] = '흥제{}동'.format(i)
    pop.loc[pop['행정구역']=='흥{}동'.format(i),'행정구역'] = '흥제{}동'.format(i)

In [ ]:
# trash,pop 면목제3.8동 -> 면목3.8동
trash.loc[trash['행정구역']=='면목제3.8동','행정구역'] = '면목3.8동'

pop.loc[pop['행정구역']=='면목제3.8동','행정구역'] = '면목3.8동'


In [ ]:
seoul_df = pd.merge(trash,pop,on='행정구역',how='left')
seoul_df = pd.merge(seoul_df,store,on='행정구역',how='left')
seoul_df = pd.merge(seoul_df,old_pop,on='행정구역',how='left')
seoul_df = pd.merge(seoul_df,pool,on='행정구역',how='left')
seoul_df = pd.merge(seoul_df,one,on='행정구역',how='left')
seoul_df = pd.merge(seoul_df,subway,on='행정구역',how='left')
seoul_df = pd.merge(seoul_df,busstop,on='행정구역',how='left')
seoul_df = pd.merge(seoul_df,e_school,on='행정구역',how='left')
seoul_df = pd.merge(seoul_df,income,on='행정구역',how='left')
seoul_df = pd.merge(seoul_df,house,on='행정구역',how='left')


In [ ]:
# 둔촌1동 기초수급 데이터 결측치 검색 후 처리
seoul_df.loc[seoul_df['행정구역']=='둔촌1동','기초수급_가구'] = 1
seoul_df.loc[seoul_df['행정구역']=='둔촌1동','기초수급_인원'] = 1

In [ ]:
#항동 제거
idx = seoul_df['행정구역'].tolist().index('항동')
seoul_df.drop(idx,axis=0,inplace=True)


In [ ]:
# '.' -> '·' 변경 (지도파일 형식 맞춤)
#ex) 종로5.6가 -> 종로5·6가
seoul_df['행정구역'] = [dong.replace('.','·') for dong in seoul_df['행정구역']]

In [ ]:
seoul_df.to_csv('서울시_최종.csv',encoding='euc-kr',index=False)